In [1]:
import os
import requests
import scipy
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [2]:
print("PyDEseq2:", pydeseq2.__version__)
print("Scipy:", scipy.__version__)
print("Numpy:", np.__version__)
print("Pandas:", pd.__version__)
print(dir(pydeseq2))

PyDEseq2: 0.4.12
Scipy: 1.14.1
Numpy: 1.24.4
Pandas: 1.5.3
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'dds', 'default_inference', 'ds', 'grid_search', 'inference', 'preprocessing', 'utils']


In [3]:
def Tscore(df, experimentalcols, controlcols, output_subfolder):
    dfstat = pd.DataFrame()
    dfpval = pd.DataFrame()
    tempnum = 0
    row_header = df.iloc[:, 0]
    midpt = int((len(df.index)) / 2)
    for r in df.iterrows():
        row = r[1]
        controlvals = []
        experimentalvals = []
        for iter in controlcols:
            controlvals.append(row[iter])
        for iter in experimentalcols:
            experimentalvals.append(row[iter])

        stat, pval = scipy.stats.ttest_ind(experimentalvals[1:], controlvals[1:], equal_var=False)
        tempstatseries = pd.Series(stat)
        temppvalseries = pd.Series(pval)        
        dfstat = pd.concat([dfstat, tempstatseries])
        dfpval = pd.concat([dfpval, temppvalseries])

        if tempnum % 1000 == 0:
            print('T-scoring gene #' + str(tempnum))
        tempnum = tempnum + 1
       
    #calculate log10 of pvalues
    df2 = pd.concat([dfstat, dfpval], axis = 1)
    df2.columns = ['Tscore', 'pval']
    oldpval = df2['pval']  
    newpval = []
    for i in oldpval:
        tempnewpval = np.log10(i)
        tempnewpval = tempnewpval * (-1)
        newpval.append(float(tempnewpval))
    df2['log10pval'] = newpval
    dfstat = df2['log10pval']
     
    #save gene signature data
    df2.set_index(row_header, inplace=True)
    save_loc = output_subfolder + "\Tscoredata.txt"
    df2.to_csv(save_loc, sep='\t', index=True)
    print('Tscore done')
    print()
    return dfstat

In [4]:
def FCstat(df, experimentalcols, controlcols, output_subfolder):
    dfFC = pd.DataFrame()
    tempnum = 0
    row_header = df.index
    for r in df.iterrows():
        geneID = r[0]
        row = r[1]
        controlvals = []
        experimentalvals = []
        for iter in controlcols:
            controlvals.append(row[iter])
        for iter in experimentalcols:
            experimentalvals.append(row[iter])
        
        m1 = np.mean(experimentalvals[1:])
        m2 = np.mean(controlvals[1:])
        FC = m1 - m2
        tempFC = pd.Series(FC, name = geneID)        
        dfFC = pd.concat([dfFC, tempFC])

        if tempnum % 1000 == 0:
            print('FC of gene #' + str(tempnum))
        tempnum = tempnum + 1
    dfFC = dfFC.reset_index(drop=True)    

    #save gene signature data    
    dfFC2 = dfFC
    dfFC2.set_index(row_header, inplace=True)
    save_loc = output_subfolder + "\FCdata.txt"
    dfFC2.to_csv(save_loc, sep='\t', index=True, header=False) #change file name as needed
    print('FC done')
    print()
    return dfFC

In [5]:
def volcano_plot(df, log10pvaldf, dfFC, output_subfolder):
    besthighFClist = []
    bestlowFClist = []
    normalFClist = []
    besthighpvallist = []
    bestlowpvallist = []
    normalpvallist = []
    besthighpos = []
    bestlowpos = []

    pnotFCFClist = []
    pnotFCpvallist = []
    FCnotpFClist = []
    FCnotppvallist = []

    tempFCarray = dfFC[0]
    tempnum = 0

    for i in log10pvaldf:
        i2 = tempFCarray[tempnum]
        if i > 1: #p<0.05
            if i2 > 1: #FC for top genes
                besthighFClist.append(i2)
                besthighpvallist.append(i)
                besthighpos.append(tempnum)
            elif i2 < -1: #FC for bottom genes
                bestlowFClist.append(i2)
                bestlowpvallist.append(i)
                bestlowpos.append(tempnum)
            else:
                pnotFCFClist.append(i2)
                pnotFCpvallist.append(i)
        else:
            if i2 > 1 or i2 < -1: #FC of 2 for top and bottom genes
                FCnotpFClist.append(i2)
                FCnotppvallist.append(i)
            else:
                normalFClist.append(i2)
                normalpvallist.append(i)
        tempnum = tempnum + 1

    top = []
    for i in besthighpos:
        r = df.iloc[[i]]
        geneID = list(r.iloc[:, 0])
        top.append(geneID)

    bottom = []
    for i in bestlowpos:
        r = df.iloc[[i]]
        geneID = list(r.iloc[:, 0])
        bottom.append(geneID)

    print('There are', (len(top)) , 'positive genes.')
    temp_df = pd.DataFrame(top, columns=['Positive Genes'])
    save_loc = output_subfolder + "\\positivegenes.txt"
    temp_df.to_csv(save_loc, sep='\t', index=True, header=False) #change file name as needed

    print('There are', (len(bottom)) , 'negative genes.')
    temp_df = pd.DataFrame(bottom, columns=['Negative Genes'])
    save_loc = output_subfolder + "\\negativegenes.txt"
    temp_df.to_csv(save_loc, sep='\t', index=True, header=False) #change file name as needed

    #plot results
    plt.figure(figsize=(10, 10))
    plt.scatter(besthighFClist, besthighpvallist, label = 'p-value and log2FC', color = 'red', marker = 'p')
    plt.scatter(bestlowFClist, bestlowpvallist, color = 'red', marker = 'p')
    plt.scatter(FCnotpFClist, FCnotppvallist, label = 'log2FC', color = 'green', marker = 's')
    plt.scatter(pnotFCFClist, pnotFCpvallist, label = 'p-value', color = 'blue', marker = 'x')
    plt.scatter(normalFClist, normalpvallist, label = 'not significant', color = 'black', marker = 'o')
    plt.axhline(y=1, color='navy', linestyle='--', label = 'cut-off')
    plt.axvline(x=1, color='navy', linestyle='--')
    plt.axvline(x=-1, color='navy', linestyle='--')
    plt.title("Volcano plot")
    plt.xlabel("log2(fold change)")
    plt.ylabel("-log10(p-value)")
    plt.legend(loc='best')
    save_loc = output_subfolder + "\\volcanoplotfig.png"
    plt.savefig(save_loc, dpi=300)
    plt.show()

#### Main Program

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
cwd = os.getcwd()
input_folder = cwd + "\input" # Define the input folder containing the .txt files
output_folder = cwd + "\output" # Define the output folder
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [8]:
for filename in os.listdir(input_folder):
    if filename.endswith("data.txt"):
        # Load the sample file
        samplefilename = filename.replace("data", "samples")
        sample_path = os.path.join(input_folder, samplefilename)
        raw_samples_df = pd.read_csv(sample_path, low_memory=False, delimiter="\t", header=None)

        # Extract conditions from the first row
        raw_samples_df.iloc[0] = pd.to_numeric(raw_samples_df.iloc[0], errors='coerce')
        conditions = raw_samples_df.iloc[0].map({1: 'diseased', 0: 'control'})        

        # Create sample names
        sample_ids = [f"sample{i+1}" for i in range(raw_samples_df.shape[1])]

        # Create the metadata DataFrame
        your_metadata = pd.DataFrame({'condition': conditions.values}, index=sample_ids)

        
        # Load the data file
        data_path = os.path.join(input_folder, filename)
        raw_data_df = pd.read_csv(data_path, low_memory=False, delimiter="\t")
        
        # Pre-process data: keep rows where all values are >=10
        rows_data_df = raw_data_df[(raw_data_df.iloc[:, 1:] >= 10).sum(axis=1) >= 6]  # At least 6 samples with counts >= 10
        rows_removed = raw_data_df.shape[0] - rows_data_df.shape[0]
        print(f"Number of rows removed: {rows_removed}")        

        # Rename columns to match metadata
        rows_data_df.columns = ['Gene Symbol'] + your_metadata.index.tolist()

        # Separate gene symbols and count matrix
        gene_symbols = rows_data_df.iloc[:, 0]
        count_matrix = rows_data_df.iloc[:, 1:]

        # Ensure count matrix is numeric
        count_matrix = count_matrix.apply(pd.to_numeric, errors='coerce')
        if count_matrix.isna().any().any():
            print("Found NaN values in count matrix. Removing invalid rows...")
            count_matrix = count_matrix.dropna()
        count_matrix = count_matrix.astype(int)

        count_matrix = count_matrix.T

        
        # Create DeseqDataSet
        data_df = DeseqDataSet(
            counts=count_matrix,
            design_factors="condition",
            metadata=your_metadata
        )
        
        # Run DESeq2 analysis
        data_df.deseq2()  # This includes normalization, dispersion estimation, and fitting
        
        # Perform statistical testing and log-fold change shrinkage
        deseq_stats = DeseqStats(data_df)  # Create a DeseqStats object
        deseq_stats.lfc_shrink()  # Apply log2 fold-change shrinkage
        
        # Access the results
        results = deseq_stats.results_df  # Results DataFrame with shrunken log fold changes
        results = results.T  # Transpose if needed for your downstream analysis
        results.index = gene_symbols  # Reassign gene symbols if necessary

        # Access the results
        #FC_mod_results = data_df.to_df()
        #FC_mod_results = FC_mod_results.T
        #FC_mod_results.index = gene_symbols
        
        # establish variable for dataset name (e.g., GSEXXXXX)
        datasetname = filename.replace("data.txt", "")
        print(datasetname)
        
        output_subfolder = output_folder + "\\" + datasetname #define output folder
        if not os.path.exists(output_subfolder):
            os.mkdir(output_subfolder)        
        
        # Find the corresponding sample file
        sample_filename = filename.replace("data", "samples")
        sample_path = os.path.join(input_folder, sample_filename)

        # Ensure the sample file exists before loading
        if os.path.exists(sample_path):
            sample_df = pd.read_csv(sample_path, sep="\t", header=None)

            # Assign experimental and control columns based on sample data
            experimentalcolsarray = sample_df.iloc[0].to_numpy() == 1
            experimentalcols = np.where(experimentalcolsarray)[0].tolist()
            print("Experimental")
            for l in experimentalcols:
                print(raw_data_df.columns[l+1]) #check this output to ensure proper column selection
            print()
            
            controlcolsarray = sample_df.iloc[0].to_numpy() == 0
            controlcols = np.where(controlcolsarray)[0].tolist()
            print("Control")
            for l in controlcols:
                print(raw_data_df.columns[l+1]) #check this output to ensure proper column selection
            print()
                        
            #analyze data
            count_matrix = count_matrix.T
            log10pvaldf = Tscore(count_matrix, experimentalcols, controlcols, output_subfolder) #call function to calculate log10pval

            dfFC = FCstat(FC_mod_results, experimentalcols, controlcols, output_subfolder) #call function to calculate log10pval
            
            volcano_plot(raw_data_df, log10pvaldf, dfFC, output_subfolder) #call function to analyze and visualize volcano plot
            
        else:
            print(f"No matching sample file for {filename}")        
        
        

Number of rows removed: 2488


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.72 seconds.

Fitting dispersion trend curve...
... done in 0.19 seconds.

Fitting MAP dispersions...
... done in 0.76 seconds.

Fitting LFCs...
... done in 0.56 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 0 outlier genes.



AttributeError: 'DeseqStats' object has no attribute 'SE'